In [7]:
import pandas as pd
import numpy as np
import os
from datetime import *
from fn import *

def o_print(my_dict):
    for key in my_dict.items():
        x = my_dict.get(key)

def getvalue(my_dict, ky):
    if ky is not None:
        for key, value in my_dict.items ():
            if key in str (ky):
                return value
        else:
            return 0

TS = lambda x : '2G' if ('2G SITE DOWN' in x) \
                else ('3G' if ('3G SITE DOWN' in x) \
                else ('4G' if ('4G SITE DOWN' in x) \
                else ('MF' if ('MAIN' in x) \
                else ('DL' if ('VOLTAGE' in x) \
                else ('TM' if ('TEMPERATURE' in x) \
                else ('SM' if ('SMOKE' in x) \
                else ('GN' if ('GEN' in x) \
                else ('GN' if ('GENSET' in x) \
                else ('TH' if ('THEFT' in x) \
                else ('C2G' if ('2G CELL DOWN' in x) \
                else ('C3G' if ('3G CELL DOWN' in x) \
                else ('C4G' if ('4G CELL DOWN' in x) \
                else ('DOOR' if ('DOOR' in x) \
                else "NA")))))))))))))

dfd = pd.read_csv (os.getcwd() + "\\OMDB.csv")
lss = dfd['sCode'].to_list()

def codecorr(code,akey):
    cd = code
    if 'UNKNOW' in code:
        for i in range(len(lss)):
            vl = akey.find(lss[i])
            if vl > 0 and vl is not None:
                cd = akey[vl:vl+7]
                break
        else:
            return cd
    else:
        return cd

def msgprep_head_znwise(hd = "Periodic Notification"):
    nw = datetime.now()
    dt = nw.strftime("%d-%m-%Y")
    tm = nw.strftime("%H:%M")
    a1 = hd + " at " + tm + " on " + dt
    return a1

def catmap_mod(df,dfdb,dfp1p2):
    df0 = df.rename (columns=str.upper)
    ls = ['NODE','RESOURCE','CUSTOMATTR15','SUMMARY','ALERTKEY','LASTOCCURRENCE']
    df1 = df0[ls]
    df1 = df1.assign(CAT = df1.apply (lambda x: TS (x.SUMMARY), axis=1))
    df1 = df1.assign(CODE = df1.apply (lambda x: codecorr(x.CUSTOMATTR15, x.ALERTKEY), axis=1))
    df2 = df1.assign(sCode = df1.apply (lambda x: x.CODE[0:5] if (x.CODE is not None) else "XXXXXXXX", axis=1))
    df3 = df2.merge (dfdb, on='sCode')
    df3['CODECAT'] = df3['CUSTOMATTR15'].str.cat(df3['CAT'])
    df3['ZNCAT'] = df3['sZone'].str.cat(df3['CAT'])
    df3 = df3.assign(CDLO = df3.apply (lambda x: x['CUSTOMATTR15'] + ": " + x['LASTOCCURRENCE'], axis=1))
    df4 = df3.merge (dfp1p2, on='CUSTOMATTR15')
    print(df4.columns)
    return df4

def rmvdup(df,lscol=[]):
    df1 = df.drop_duplicates(subset=lscol, inplace=False, ignore_index=True)
    return df1

def zonewise_count(df0, oncat=[]):
    zn = ['DHK_M','DHK_N','DHK_O','DHK_S','CTG_M','CTG_N','CTG_S','COM','NOA','BAR','KHL','KUS','MYM','RAJ','RANG','SYL']
    Tcnt = {}
    hd1 = ""
    hd2 = ""
    for j in range(len(oncat)):
        Tcnt[oncat[j]] = 0
        if hd1 == "":
            hd1 = "Region: " + oncat[j]
        else:
            hd1 = hd1 + "/" + oncat[j]
    hp = chr(10)
    ls = []
    for i in range(len(zn)):
        for n in range(len(oncat)):
            zct  = zn[i] + oncat[n]
            cnt = countifs(df0,df0['ZNCAT'],zct)
            ls.append(str(cnt))
            current_val = int(getvalue(Tcnt, oncat[n])) + cnt
            Tcnt[oncat[n]] = current_val
        else:
            hp = hp + chr(10) + zn[i] + ": " + "/".join(list(ls))
            ls = []
    for k in range(len(oncat)):
        hdval = Tcnt.get(oncat[k])
        if hd2 == "":
            hd2 = "National: " + str(hdval)
        else:
            hd2 = hd2 + "/" + str(hdval)
    else:
        trail1 = "This is RPA generated periodic notification." + chr(10) + "For any query, please contact with " + chr(10)
        trail2 = trail1 + "SOC Shift Manager, 01817183680"
        FinalText = msgprep_head_znwise() + chr(10) + chr(10) + hd1 + chr(10) + hd2 + hp + chr(10) + chr(10) + trail2
        return FinalText

def zonewise_dclow(df0, pickcols=['CUSTOMATTR15','LASTOCCURRENCE'], cols1="CAT", cols1_val="DL", cols2=False, ls_val_cols2=False):
    #parse by with max 2 condition zonewise
    zn = ['DHK_M','DHK_N','DHK_O','DHK_S','CTG_M','CTG_N','CTG_S','COM','NOA','BAR','KHL','KUS','MYM','RAJ','RANG','SYL']
    Tcnt = {}
    heap = ""
    for i in range(len(zn)):
        hp = ""
        znst = zn[i]
        ndf = df0[df0[cols1].str.contains(cols1_val) & df0['sZone'].str.contains(znst)]
        if ndf.shape[0] != 0:
            cnt_in_zn = znst + ": " + str(ndf.shape[0])
            ndf = ndf.astype(str)
            xdf = ndf[pickcols]
            fdf = xdf.assign(FCOL = xdf.apply(' - '.join, axis=1))
            fdf = fdf.reset_index()
            for n in range(fdf.shape[0]):
                print(n)
                if hp == "":
                    hp = cnt_in_zn + chr(10) + fdf.loc[n ,'FCOL']
                else:
                    hp = hp + chr(10) + fdf.loc[n ,'FCOL']
            else:
                if heap == "":
                    heap = hp
                else:
                    heap = heap + chr(10) + chr(10) + hp
    else:
        print(heap)
        
def zonewise_parse(df1, whichzn, pickcols=[], colsMain="CAT", colsMain_val=['DL'], cols2=False, ls_val_cols2=False):
    heap = ""
    if df1.shape[0] != 0:
        if len(pickcols) is not None:
            for i in range(len(pickcols)):
                df1 = df1.assign(COLO1 = "0")
                df1['CDLO1'] = df1.apply (lambda x: x['CDLO'] + " - " + x[pickcols[i]], axis=1)
                df1['CDLO'] = df1['CDLO1']
                df1 = df1.drop(['CDLO1'], axis=1)
        if df1.shape[0] != 0:
            for i in range(len(colsMain_val)):
                hp1 = ""
                cri_1 = colsMain_val[i]
                df2 = df1[df1[colsMain].isin([cri_1])]
                if df2.shape[0] !=0 and ls_val_cols2 != False:
                    for j in range(len(ls_val_cols2)):
                        cri_2 = ls_val_cols2[j]
                        dff = df2[df2[cols2].isin([cri_2])]
                        if dff.shape[0] != 0:
                            dfx = dff.reset_index()
                            hp = cri_2 + "| " + str(dff.shape[0]) + chr(10) + dfx['CDLO'].str.cat(sep=chr(10))
                            if hp1 == "":
                                hp1 = hp
                            else:
                                hp1 = hp1 + chr(10) + chr(10) + hp
                    else:
                        heap = heap + chr(10) + chr(10) + cri_1 + " || Count: " + str(df2.shape[0]) + chr(10) + chr(10) + hp1
                else:
                    if df2.shape[0] !=0:
                        df2 = df2.reset_index()
                        hp = cri_1 + ": " + str(df2.shape[0]) + chr(10) + df2['CDLO'].str.cat(sep=chr(10))
                        if heap == "" or heap== chr(10):
                            heap = hp
                        else:
                            heap = heap + chr(10) + chr(10) + hp
                    else:
                        heap = heap + chr(10) + cri_1 + ": " + "NA"
            else:
                finalout = "Region: " + whichzn + chr(10) + heap
                return finalout
                
def parsing(df, whichzn, pickcols=[], colsMain="CAT", colsMain_val=['DL'], cols2=False, ls_val_cols2=False):
    zn = {'DHK_M':'','DHK_N':'','DHK_O':'','DHK_S':'','CTG_M':'','CTG_N':'','CTG_S':'','COM':'','NOA':'',
          'BAR':'','KHL':'','KUS':'','MYM':'','RAJ':'','RANG':'','SYL':''}
    cnt = 0
    rval = ""
    if whichzn =="ALL":
        for ky in zn.keys():
            cnt = cnt + 1
            whichzn = ky
            df1 = df[df['sZone'].isin([ky])]
            if ls_val_cols2 != False:
                zn[ky] = zonewise_parse(df1,whichzn,pickcols,colsMain, colsMain_val, cols2, ls_val_cols2)
                print(zn.get(ky))
                print("############################",chr(10))
            else:
                zn[ky] = zonewise_parse(df1,whichzn,pickcols,colsMain, colsMain_val)
                print(zn.get(ky))
                print("############################",chr(10))
        else:
            return zn
    elif whichzn =="":
        if ls_val_cols2 != False:
            rval = zonewise_parse(df,whichzn,pickcols,colsMain, colsMain_val, cols2, ls_val_cols2)
        else:
            rval = zonewise_parse(df,whichzn,pickcols,colsMain, colsMain_val)
        return rval
    elif whichzn !="NA" and whichzn!="ALL" and whichzn !="":
        df1 = df[df['sZone'].isin([whichzn])]
        if ls_val_cols2 != False:
            rval = zonewise_parse(df1,whichzn,pickcols,colsMain, colsMain_val, cols2, ls_val_cols2)
        else:
            rval = zonewise_parse(df1,whichzn,pickcols,colsMain, colsMain_val)
        return rval
                
            
            
            
    #df2 = df1[df1[cols1].str.contains(cols1_val) & df0['sZone'].str.contains(znst)]
    #df1 = df.assign(LOCD = xdf.apply(' - '.join, axis=1))
    #pickcols.insert(0, 'sZone')
    #pickcols.insert(0, 'CAT')
    #ndf = df0[df0[cols1].str.contains(cols1_val) & df0['sZone'].str.contains(znst)]
        
    
#sclick.csv must have Required column : 'NODE','RESOURCE','CUSTOMATTR15','SUMMARY','ALERTKEY','LASTOCCURRENCE'
df = pd.read_csv(os.getcwd() + "\\sclick.csv")  # data source, 
dfdb = pd.read_csv (os.getcwd() + "\\OMDB.csv") #fixed data in same folder
dfp1p2 = pd.read_csv (os.getcwd() + "\\site_p1p2.csv")
odf = catmap_mod(df,dfdb,dfp1p2) # function is for processing data
#ST = zonewise_count(xx, ['2G','3G','4G'])  #2G,3G,4G derived from lambda function "TS". check abov
#result = parsing(odf,whichzn="ALL",pickcols=['Name'],colsMain="CAT", colsMain_val=['DL','MF'], cols2="ATRB", ls_val_cols2=['eco','Ulka'])
#result = parsing(odf,whichzn="COM",pickcols=['Name'],colsMain="CAT", colsMain_val=['DL','MF'], cols2="ATRB", ls_val_cols2=['eco','Ulka'])
result = parsing(odf,whichzn="",pickcols=['Name'],colsMain="CAT", colsMain_val=['2G','3G','4G'])
try:
    print(result)
except:
    o_print(result)




Index(['NODE', 'RESOURCE', 'CUSTOMATTR15', 'SUMMARY', 'ALERTKEY',
       'LASTOCCURRENCE', 'CAT', 'CODE', 'sCode', 'sZone', 'TCOUNT', 'CODECAT',
       'ZNCAT', 'CDLO', 'P1P2', 'ATRB', 'Name'],
      dtype='object')
Region: 
2G: 23
CXUKH72: 28/12/2020 6:24 - Maricha Palong 3, Haldia Palong
DHSVR41: 28/12/2020 22:29 - Bhorodeshi
DHJTB66: 23/12/2020 4:43 - SHAHID TITUMIR ROAD
DHSBB32: 20/12/2020 2:10 - East Basabo
DHDHN92: 29/12/2020 2:11 - IBS_Roxy Paints
DHPLB19: 28/12/2020 15:43 - Mirpur 12E
NOSNM13: 29/12/2020 2:29 - Bogadia
CXCKR02: 28/12/2020 12:45 - Chokoria
CGPRT48: 29/12/2020 1:29 - Narikal Tola
CGPRT47: 29/12/2020 2:17 - Mabud Villa
CGPRT36: 29/12/2020 2:32 - Nayahat 2
CGSKN32: 29/12/2020 1:55 - Golar Ghat Bazar
CGSKN51: 29/12/2020 1:55 - SATKANIA2_Reloc_Airtel
CGPTG52: 28/12/2020 21:49 - Hoque Villa , Katgor
CGPTG52: 28/12/2020 21:49 - Hoque Villa , Katgor
CGPTG52: 29/12/2020 1:56 - Hoque Villa , Katgor
CGPTG12: 28/12/2020 23:38 - Nurjamal_Sarak
CGPTG35: 29/12/2020 1:46 - Kazi